# WiMLDS Zürich: Sneak peek at Geodata

SIT Academy, 18.05.2022

Authors: WiMLDS Zurich // Litix // DIA 

You can use content from this notebook in your own analyses and projects. If you do, please drop us a message: zurich@wimlds.org, teresa@litix.ch or gubicza_agi@yahoo.co.uk

In [ ]:
#import packages

from ipyleaflet import Map, WMSLayer, basemaps, projections

## Practice 1: load and transform

In this exercise we load and transform different kinds of geodata.

#### Step 1: load background map

Create a base map:

In [ ]:
# create a base map with OpenStreetMaps
m = Map(center=(47.350, 8.560), zoom=15)

# show the interactive raster map
m

Load background map (WMS)

In [ ]:
# add a layer on top, for example orthophotos:
m = Map(center=(47.350, 8.560), zoom=15)

# Orthofoto Zürich
wms1 = WMSLayer(
    url='http://wms.zh.ch/OGDOrthoZH',
    layers='OGDOrthoZH',
    format='image/png',
    transparent=True,
    attribution='GIS-ZH'
)

m.add_layer(wms1)

m

The following shows other sources which we can use as layers:

In [ ]:
# other sources:
m = Map(center=(47.350, 8.560), zoom=15)


# OverviewMap
wms2 = WMSLayer(
    url='http://wms.zh.ch/upwms/',
    layers='upwms',
    format='image/png',
    transparent=False,
    attribution='GIS-ZH'
)

# Swissimage
wms3 = WMSLayer(
    url='https://wms.geo.admin.ch/',
    layers='ch.swisstopo.swissimage',
    format='image/png',
    transparent=True,
    attribution='swisstopo'
)

# Pixelkarte
wms4 = WMSLayer(
    url='https://wms.geo.admin.ch/',
    layers='ch.swisstopo.pixelkarte-farbe',
    format='image/png',
    transparent=True,
    attribution='swisstopo'
)

# wms3, wms4 are full maps, but wms2 can also be transparent

m.add_layer(wms2)
m

#### Step 2: Load bus stops, roads and lakes

Option 1: Overlay on the raster layer with additional raster layers

In [ ]:
# do we always need to recreate the m? 
m = Map(center=(47.350, 8.560), zoom=15)

# Public transport stops
wms1 = WMSLayer(
    url='https://wms.geo.admin.ch/',
    layers='ch.bav.haltestellen-oev',
    format='image/png',
    transparent=True,
    attribution='swisstopo'
)

# Streets
wms2 = WMSLayer(
    url='https://wms.geo.admin.ch/',
    layers='ch.swisstopo.swisstlm3d-strassen',
    format='image/png',
    transparent=True,
    attribution='swisstopo'
)

# Rivers and lakes
wms3 = WMSLayer(
    url='https://wms.geo.admin.ch/',
    layers='ch.swisstopo.swisstlm3d-gewaessernetz',
    format='image/png',
    transparent=True,
    attribution='swisstopo'
)


m.add_layer(wms3)
m.add_layer(wms2)
m.add_layer(wms1)

# note the order of the layers! what do you think, what is the reason for this order? Try to rearrange and / or
# change the layer transparency from True to False. what do you observe?
# 5 min to play with the order of layers 

m



#### Option 2: We can also use vector layers

In [ ]:
# import some useful packages

from owslib.wfs import WebFeatureService
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

In [ ]:
# until now we used the wMs -> map services, now wFs -> feature layers 
# additional info: about every object on the layer 

# public transport stops

# URL for WFS backend
url = "http://maps.zh.ch/wfs/HaltestellenZHWFS"

# Initialize
wfs = WebFeatureService(url=url)

# Service provider 
print(wfs.identification.title)

# Get WFS version
print(wfs.version)

# Available methods
print([operation.name for operation in wfs.operations])

# Available data layers
# there can be more layers than 1, just here we have 1 
print(list(wfs.contents))

In [ ]:
# dir(wfs)

Link to the dataset: http://www.geolion.zh.ch/geodatensatz/883

In [ ]:
# Get data from WFS
# Specify the parameters for fetching the data
layer = 'haltestellen'
params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson')

# Parse the URL with parameters
# only use it to create the url
q = Request('GET', url, params=params).prepare().url

# geopandas can read directly from a URL
stops = gpd.read_file(q) 

In [ ]:
# stops is the resulting geodataframe with geo and symbiological information 
# stops.iloc[0]
stops.head()

In [ ]:
stops.symb_text.unique() # eg stop categories are a type of symbiology

In [ ]:
# try to change the color coding! use the parameter "column"
stops.plot(column = stops.symb_text, kind = 'geo', figsize = (15,15), legend = True) 

In [ ]:
# The previous example showed points on a vector layer, let's see some lines (streets):
# URL for WFS backend
url = "http://maps.zh.ch/wfs/TBAStrZHWFS"
# how to get these? 

# Initialize
wfs = WebFeatureService(url=url)

# Get data from WFS
# Specify the parameters for fetching the data
layer = 'strassenachsen'
params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson')

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

streets = gpd.read_file(q)


##### Can you plot the streets? 

In [ ]:
#... here goes your code

*Answer hidden below:* 

In [ ]:
streets.plot(column = streets.strasstyp, kind = 'geo', figsize = (20,20), legend = True)

In [ ]:
# Let's plot polygons (here the ground cover of buildings)

# URL for WFS backend
url = "http://maps.zh.ch/wfs/AVZHWFS"

# Initialize
wfs = WebFeatureService(url=url)

# Get data from WFS
# Specify the parameters for fetching the data
# Also specify a bounding box (2km x 2km) in order to get less data
layer = 'bodenbedeckung_f'
bbox="2684000,1244000,2686000,1246000" 
# how to find the bbox in advance? 
# maps.zh.ch, look at the mouse coords in bottom left 
# 2692824 / 1247783 -> horz / vertical 
# swiss coordinate system, 1 unit is one meter, 2686000-2684000 = 2km
# only the data that touch this box is delivered -> they can go out of the box too -> look at the blue polygons in bottom left corner 
# you really want to do it, b/c otherwise you will download all the data 

params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson', bbox=bbox)

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

buildings = gpd.read_file(q)

In [ ]:
buildings.iloc[0]

In [ ]:
buildings.art.unique()

In [ ]:
# what is in the geodata column?
buildings.iloc[0].geometry

##### Can you plot the polygons? 

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
buildings.plot(column = buildings.art, kind = 'geo', figsize = (20,20), legend = True)

##### Can you overlay the streets with the stops?

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# Let's overlay the streets with the stops:
base = streets.plot(column = streets.strasstyp, figsize = (20,20))

stops.plot(ax=base, column = stops.symb_code, markersize=7);

##### Can you overlay the bulidings and the roads? Do they match? Can you zoom in?

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# Solution:
base = buildings.plot(color='white', edgecolor='black', figsize = (20,20))

streets.plot(ax=base, column = streets.strasstyp)
base.set_xlim(2.683e6, 2.688e6)
base.set_ylim(1.2425e6, 1.247e6)

### NOTE!!! You can save your dataframes by pickling them, writing to excel, etc. and you can save your visuals as raster images. The methods familiar from working with pandas dataframes and matplotlib plots work here as well.


In [ ]:
buildings.to_csv('tmp.csv')

# Practice 2: go to the 3rd dimension

In this exercise we determine the height level of each public transport stops and calculate which stations will go under water in case the level of Lake Zürich increases significantly.

In [ ]:
from owslib.wfs import WebFeatureService
import geopandas as gpd
from requests import Request
import requests
from owslib.wfs import WebFeatureService

In [ ]:
# First let's query the locations of the stops close to the lake:

url = "http://maps.zh.ch/wfs/HaltestellenZHWFS"
layer = 'haltestellen'
bbox="2680000,1230000,2710000,1250000"
params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson', bbox=bbox)

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

stops = gpd.read_file(q)

In [ ]:
stops.shape

In [ ]:
# we can access the properties of the geoobject:
stops["geometry"].iloc[0].x 

In [ ]:
# A way to query the height of a point given by x_loc and y_loc coordinates is the following:

x_loc = 2685834 
y_loc = 1244093
url = "https://api3.geo.admin.ch/rest/services/height?easting=" + str(x_loc) + "&northing=" + str(y_loc)
r = requests.get(url)
height_of_point = r.json()["height"]



##### Can you write a cycle to query the level of all stops? How would you determine the height of the lake?

In [ ]:
stops.loc[0,'geometry']

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
%%time 

# Solution:

# level of stops:
heights = []
for row in stops.iterrows():
    url = "https://api3.geo.admin.ch/rest/services/height?easting=" + str(row[1]["geometry"].x) + "&northing=" + str(row[1]["geometry"].y)
    r = requests.get(url)
    heights.append(r.json()["height"])
stops["Height"]=heights



*Answer hidden below:* 

In [ ]:
# Solution

# level of the lake: 
# using the maps from the previous exercises,using any coordinate pairs 
# corresponding to the surface of the lake will give the same result (405.9 m)

url = "https://api3.geo.admin.ch/rest/services/height?easting=" + str(2684500.0) + "&northing=" + str(1245000.0)
r = requests.get(url)
height_of_lake = r.json()["height"]
height_of_lake

##### Now that we know the height of all stops around the lake, let's calculate which one would go under water if the level of the lake increases by 2 meters.

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# Solution:

# calculate, if they become flooded:
stops["flooded"] = stops["Height"].astype(float) < 407.9
print("Number of stations under water at a lake height of 407.9 m: " + str(stops["flooded"].sum()))

# visualize

stops.plot(column = stops.flooded, figsize = (15,15)) 

___

Bonus exercise (without solution): 
Determine the height profile of the city of Zurich.
Solution steps: 
1. determine an x-y grid covering the area of the city. 
2. Write a cycle to query the height of all grid points. 
3. Visualize the height level on a colormap and/or on a 3d plot.

In [ ]:
# ... here goes your code!
# There is no solution :) 

# Practice 3: points within polygons

Let's look at the previous problem in a different way. It is possible to query which areas are under danger of flooding. So we can load the flooding hazard map of the city and then see which stops are in one of the hazardous regions.

In [ ]:
# This is how to load the hazard map

# Get data from WFS
# Specify the parameters for fetching the data
url = "http://maps.zh.ch/wfs/AwelGKHWZHWFS"
layer = 'hw_gk'
bbox="2684000,1244000,2686000,1246000"
params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson', bbox=bbox)

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

gk = gpd.read_file(q)

In [ ]:
gk.head()

##### By checking the gk dataframe (gk or gk.head()), you can see that the hazardous areas are polygons. Do you know how to assess, if the stops are located within any of these polygons?

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# Solution:

indanger = []

for row in stops.iterrows():
    indanger.append(gk.geometry.contains(row[1].geometry).any())

stops["in_danger"] = indanger 
base = gk.plot(figsize = (20,20))

stops.plot(ax=base, column = stops.in_danger, markersize=100, cmap='coolwarm')
base.set_xlim(2.6837e6, 2.6865e6)
base.set_ylim(1.2440e6, 1.2465e6)

##### What are the names of endangered stops?

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
stops[stops['in_danger']==1]

## Bonus exercise:

##### 1. Calculate the number of stops per square km in a community (gemeinde) and visualize it on a choropleth plot.

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# Solution:

# query the community borders:

# URL for WFS backend
url = "http://maps.zh.ch/wfs/AVZHWFS"

# Initialize
wfs = WebFeatureService(url=url)

layer = 'gemeinden_f'
params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson')

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

communities = gpd.read_file(q)

communities.head()

##### 2. Get all the stops again

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# Get all the stops again

url = "http://maps.zh.ch/wfs/HaltestellenZHWFS"
layer = 'haltestellen'
params = dict(service='WFS', version="1.0.0", request='GetFeature',
      typeName=layer, outputFormat='geojson')

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

stops = gpd.read_file(q)

##### 3. Calculate the number of stops in each community:

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# calculate the number of stops in each community:

nr_of_stops = []

for row in communities.iterrows():
    nr_of_stops.append(stops.geometry.within(row[1].geometry).sum())

communities["nr_of_stops"] = nr_of_stops

##### 4. Calculate the area of each community

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
# calculate the area of each community

surface = communities['geometry'].to_crs({'proj':'cea'})
communities["area (km2)"] = surface.area / 10**6

### 5. Calculate the density 

In [ ]:
# ... here goes your code!

*Answer hidden below:* 

In [ ]:
communities["stop density"] = communities["nr_of_stops"]/communities["area (km2)"]

communities.plot(column = communities["stop density"], cmap = 'OrRd', figsize = (15,15))

That's it! We hope you had fun :) 